<a href="https://colab.research.google.com/github/amilyk/HappyAgents/blob/main/AutoGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
import asyncio
_ = load_dotenv(find_dotenv())

In [2]:
# 安装 autogen‑agentchat（核心对话框架）
!pip install -U autogen-agentchat

# 安装 autogen‑ext 并带上 OpenAI 客户端（最常用）
!pip install -U "autogen-ext[openai]"


In [3]:
!touch .env
!echo "DEEPSEEK_API_KEY='你的api-key'" > .env

# 软件开发团队

In [5]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

def create_openai_model_client():
    """创建 OpenAI 模型客户端用于测试"""
    return OpenAIChatCompletionClient(
            model="deepseek-chat",
            api_key=os.getenv("DEEPSEEK_API_KEY"),
            base_url="https://api.deepseek.com/v1",
            model_info={
                "function_calling": True,
                "max_tokens": 4096,
                "context_length": 32768,
                "vision": False,
                "json_output": True,
                "family": "deepseek",
                "structured_output": True,
        }
)

In [6]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent

# 1.产品经理

In [7]:
def create_product_manager(model_client):
    """创建产品经理智能体"""
    system_message = """你是一位经验丰富的产品经理，专门负责软件产品的需求分析和项目规划。

你的核心职责包括：
1. **需求分析**：深入理解用户需求，识别核心功能和边界条件
2. **技术规划**：基于需求制定清晰的技术实现路径
3. **风险评估**：识别潜在的技术风险和用户体验问题
4. **协调沟通**：与工程师和其他团队成员进行有效沟通

当接到开发任务时，请按以下结构进行分析：
1. 需求理解与分析
2. 功能模块划分
3. 技术选型建议
4. 实现优先级排序
5. 验收标准定义

请简洁明了地回应，并在分析完成后说"请工程师开始实现"。"""

    return AssistantAgent(
        name="ProductManager",
        model_client=model_client,
        system_message=system_message,
    )


# 2.软件工程师

In [8]:
def create_engineer(model_client):
    """创建软件工程师智能体"""
    system_message = """你是一位资深的软件工程师，擅长 Python 开发和 Web 应用构建。

你的技术专长包括：
1. **Python 编程**：熟练掌握 Python 语法和最佳实践
2. **Web 开发**：精通 Streamlit、Flask、Django 等框架
3. **API 集成**：有丰富的第三方 API 集成经验
4. **错误处理**：注重代码的健壮性和异常处理

当收到开发任务时，请：
1. 仔细分析技术需求
2. 选择合适的技术方案
3. 编写完整的代码实现
4. 添加必要的注释和说明
5. 考虑边界情况和异常处理

请提供完整的可运行代码，并在完成后说"请代码审查员检查"。"""

    return AssistantAgent(
        name="Engineer",
        model_client=model_client,
        system_message=system_message,
    )


# 3.代码审查员

In [9]:
def create_code_reviewer(model_client):
    """创建代码审查员智能体"""
    system_message = """你是一位经验丰富的代码审查专家，专注于代码质量和最佳实践。

你的审查重点包括：
1. **代码质量**：检查代码的可读性、可维护性和性能
2. **安全性**：识别潜在的安全漏洞和风险点
3. **最佳实践**：确保代码遵循行业标准和最佳实践
4. **错误处理**：验证异常处理的完整性和合理性

审查流程：
1. 仔细阅读和理解代码逻辑
2. 检查代码规范和最佳实践
3. 识别潜在问题和改进点
4. 提供具体的修改建议
5. 评估代码的整体质量

请提供具体的审查意见，完成后说"代码审查完成，请用户代理测试"。"""

    return AssistantAgent(
        name="CodeReviewer",
        model_client=model_client,
        system_message=system_message,
    )

# 4.用户代理

In [10]:
def create_user_proxy():
    """创建用户代理智能体"""
    return UserProxyAgent(
        name="UserProxy",
        description="""用户代理，负责以下职责：
1. 代表用户提出开发需求
2. 执行最终的代码实现
3. 验证功能是否符合预期
4. 提供用户反馈和建议

完成测试后请回复 TERMINATE。""",
    )

# 5.团队协作流程

In [11]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination


# 启动与运行

In [12]:
from autogen_agentchat.ui import Console

In [13]:
async def run_software_development_team():
  # ... 初始化客户端和智能体 ...
    print("🔧 正在初始化模型客户端...")
    model_client = create_openai_model_client()

    print("👥 正在创建智能体团队...")
    product_manager = create_product_manager(model_client)
    engineer = create_engineer(model_client)
    code_reviewer = create_code_reviewer(model_client)
    user_proxy = create_user_proxy()

     # 添加终止条件
    termination = TextMentionTermination("TERMINATE")

    # 定义团队聊天和协作规则
    team_chat = RoundRobinGroupChat(
        participants=[
            product_manager,
            engineer,
            code_reviewer,
            user_proxy
        ],
        termination_condition=termination,
        max_turns=20,
    )

    # 定义任务描述
    task = """我们需要开发一个比特币价格显示应用，具体要求如下：
            核心功能：
            - 实时显示比特币当前价格（USD）
            - 显示24小时价格变化趋势（涨跌幅和涨跌额）
            - 提供价格刷新功能

            技术要求：
            - 使用 Streamlit 框架创建 Web 应用
            - 界面简洁美观，用户友好
            - 添加适当的错误处理和加载状态

            请团队协作完成这个任务，从需求分析到最终实现。"""

    # 执行团队协作
    print("🚀 启动 AutoGen 软件开发团队协作...")
    print("=" * 60)

    # 异步执行团队协作，并流式输出对话过程
    result = await Console(team_chat.run_stream(task=task))

    print("\n" + "=" * 60)
    print("✅ 团队协作完成！")

    return result


In [14]:
result = await run_software_development_team()

print("\n📋 协作结果摘要：")
print(f"- 参与智能体数量：4个")
print(f"- 任务完成状态：{'成功' if result else '需要进一步处理'}")

🔧 正在初始化模型客户端...
👥 正在创建智能体团队...
🚀 启动 AutoGen 软件开发团队协作...
---------- TextMessage (user) ----------
我们需要开发一个比特币价格显示应用，具体要求如下：
            核心功能：
            - 实时显示比特币当前价格（USD）
            - 显示24小时价格变化趋势（涨跌幅和涨跌额）
            - 提供价格刷新功能

            技术要求：
            - 使用 Streamlit 框架创建 Web 应用
            - 界面简洁美观，用户友好
            - 添加适当的错误处理和加载状态

            请团队协作完成这个任务，从需求分析到最终实现。
---------- TextMessage (ProductManager) ----------
### 需求理解与分析
- **核心目标**：构建一个轻量级Web应用，实时展示比特币价格及24小时趋势，要求界面简洁、响应及时。
- **关键需求**：
  - 实时价格数据（USD）：需从可靠API获取最新价格。
  - 24小时趋势：包括涨跌幅百分比和具体涨跌金额。
  - 手动刷新功能：允许用户主动更新数据。
  - 错误处理：网络请求失败或数据异常时提供友好提示。
  - 加载状态：数据获取期间显示加载指示器。
- **边界条件**：无需用户登录、历史数据存储或复杂图表，重点在于实时性与可用性。

---

### 功能模块划分
1. **数据获取模块**  
   - 调用加密货币API（如CoinGecko或Binance公共API）获取实时价格和24小时变化数据。
   - 处理API响应，解析JSON数据并提取关键字段（当前价格、涨跌幅、涨跌额）。
2. **界面渲染模块**  
   - 使用Streamlit组件展示：
     - 主价格显示（大字体突出）。
     - 趋势指标（颜色区分涨跌：绿色上涨，红色下跌）。
     - 刷新按钮（触发数据重新获取）。
3. **状态管理模块**  
   - 加载状态：数据请求时显示“加载中”提示。
   - 错误状态：API失败时显示错误信息及重